In [1]:
from dypac import Dypac
import numpy as np
import sklearn as sk
import scipy as sp
import matplotlib.pyplot as plt
import time
from nilearn import datasets
from nilearn import image
from nilearn.input_data import NiftiMasker
from nilearn import plotting
from nilearn.decomposition import DictLearning, CanICA
from scipy.stats import pearsonr

ImportError: cannot import name 'dypac' from 'dypac.dypac' (/home/pbellec/git/dypac/dypac/dypac.py)

Let's start by downloading a few subjects from the ADHD200 dataset.

In [ ]:
n_subjects = 1
adhd_dataset = datasets.fetch_adhd(n_subjects)
epi_filename = adhd_dataset.func
# Also load a probabilistic grey matter segmentation
mni = datasets.fetch_icbm152_2009()

Now we run dypac. First we specify the model, with the main relevant options. Then we fit the model on the ADHD data.

In [ ]:
model = dypac(n_clusters=10, grey_matter=mni.gm, std_grey_matter=3, n_states=20, verbose=1, n_init=1, n_init_aggregation=1, n_replications=20, 
              detrend=True, smoothing_fwhm=5, standardize=True, n_batch=1, threshold_sim=0)
model.fit(epi_filename)

In [ ]:
weights = model.masker_.inverse_transform(model.weights_grey_matter_)
plotting.view_img(weights)

In [ ]:
num_comp = 17
comp = model.masker_.inverse_transform(model.components_[num_comp,:].todense())
plotting.view_img(comp, threshold=0.1, vmax=1, title="Dwell time: {dt}".format(dt=model.dwell_time_[num_comp]))

Save model in a pickle file

In [ ]:
import pickle
hf = open('model.pickle','wb')
pickle.dump(model, hf)